# CELL 0 — Cài thư viện

In [ ]:
!pip install -q transformers accelerate peft datasets bitsandbytes trl sentencepiece
!pip install -q huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 17.9 MB/s eta 0:00:00


# CELL 1 — Import thư viện

In [ ]:
import os
import json
from glob import glob
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from transformers import DataCollatorForSeq2Seq
from peft import LoraConfig, get_peft_model
import torch
from huggingface_hub import notebook_login


# CELL 2 — Kết nối Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
RAW_DIR = "/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data"
CLEAN_DIR = "/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly"
os.makedirs(CLEAN_DIR, exist_ok=True)

In [ ]:
# @title # CELL 3 — Load các file 6 cặp từ raw_data/ và merge thành 1 dataset_cleaned.jsonl

import os
import json
from glob import glob



# Các file dạng topic_01_raw_6pairs.jsonl, topic_02_raw_6pairs.jsonl...
files = sorted(glob(f"{RAW_DIR}/topic_*_raw_6pairs.jsonl"))

print("📁 Found files:", files)

all_items = []

for f in files:
    print("👉 Reading:", f)
    with open(f, "r", encoding="utf-8") as fr:
        for line in fr:
            line = line.strip()
            if not line:
                continue
            try:
                obj = json.loads(line)
                # kiểm tra format chuẩn
                if "messages" in obj and isinstance(obj["messages"], list):
                    all_items.append(obj)
            except Exception as e:
                print("❌ Parse error:", f, e)
                continue

print("📊 Tổng số hội thoại hợp lệ:", len(all_items))

# Lưu file cleaned hợp nhất
clean_file = os.path.join(CLEAN_DIR, "dataset_cleaned.jsonl")
with open(clean_file, "w", encoding="utf-8") as fw:
    for item in all_items:
        fw.write(json.dumps(item, ensure_ascii=False) + "\n")

print("✔ Đã tạo file cleaned:", clean_file)


📁 Found files: ['/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_01_raw_6pairs.jsonl', '/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_02_raw_6pairs.jsonl', '/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_03_raw_6pairs.jsonl', '/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_04_raw_6pairs.jsonl', '/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_05_raw_6pairs.jsonl', '/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_06_raw_6pairs.jsonl', '/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_07_raw_6pairs.jsonl', '/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_08_raw_6pairs.jsonl', 

In [ ]:
# @title # CELL 4 — Phân tích dataset cleaned

print("📊 Tổng hội thoại:", len(all_items))

lengths = [len(x["messages"]) for x in all_items]
avg_len = sum(lengths) / len(lengths)

print("📏 Số messages trung bình / hội thoại:", avg_len)
print("📏 Tối thiểu:", min(lengths))
print("📏 Tối đa:", max(lengths))


📊 Tổng hội thoại: 959
📏 Số messages trung bình / hội thoại: 12.0
📏 Tối thiểu: 12
📏 Tối đa: 12


# CELL 4 — Phân tích ngắn dataset

In [ ]:
print("Số mẫu:", len(all_items))
lens = [len(x["messages"]) for x in all_items]
print("Số messages trung bình:", sum(lens)/len(lens))


Số mẫu: 959
Số messages trung bình: 12.0


# CELL 5 — TRAIN VỚI LoRA

In [ ]:
# ============================
# training_v3_full_for_unsloth.ipynb
# CELL 0 — Install libs (chạy 1 lần)
# ============================
!pip install -q transformers accelerate peft datasets bitsandbytes trl sentencepiece safetensors huggingface_hub wandb


In [ ]:
# ============================
# CELL 1 — Imports + constants
# ============================
import os, json, re
from glob import glob
from pathlib import Path
from typing import List, Dict

import datasets
from datasets import load_dataset

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
)
from transformers import BitsAndBytesConfig
from peft import LoraConfig, get_peft_model

# --- Paths (chỉnh nếu cần) ---
RAW_DIR = "/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data"
CLEAN_DIR = "/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly"
os.makedirs(CLEAN_DIR, exist_ok=True)

# Model to use
MODEL_NAME = "unsloth/Llama-3.2-1B-Instruct"

# Grouping: mỗi dialogue = N_pairs (N_pairs * 2 messages)
PAIRS_PER_DIALOGUE = 6
MAX_LENGTH = 1024

print("RAW_DIR:", RAW_DIR)
print("CLEAN_DIR:", CLEAN_DIR)
print("MODEL_NAME:", MODEL_NAME)


RAW_DIR: /content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data
CLEAN_DIR: /content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly
MODEL_NAME: unsloth/Llama-3.2-1B-Instruct


In [ ]:
# ============================
# CELL 2 — Utility: extract messages robustly from file text
# - hỗ trợ multi-line JSON, pretty-print, broken lines
# - cố gắng bắt "role" & "content" bằng regex linh hoạt
# ============================
def extract_messages_from_text(text: str) -> List[Dict]:
    """
    Trích role+content từ một văn bản bất kỳ bằng regex an toàn.
    Trả về list các dict {"role":..., "content":...} theo thứ tự xuất hiện.
    """
    messages = []
    # Regex an toàn: tìm "role": "xxx" và "content": "yyy" (allow escaped quotes)
    pattern = re.compile(r'"role"\s*:\s*"([^"]+)"\s*[,}].*?"content"\s*:\s*"((?:[^"\\]|\\.)*)"', re.DOTALL)
    for m in pattern.finditer(text):
        role = m.group(1).strip()
        content = m.group(2).encode('utf-8').decode('unicode_escape')  # unescape
        content = content.replace("\n", " ").strip()
        messages.append({"role": role, "content": content})
    return messages

def load_messages_from_file(path: str) -> List[Dict]:
    """
    Nếu file là JSONL (mỗi dòng 1 object) thì parse dòng;
    else fallback đọc toàn file và extract bằng regex.
    """
    msgs = []
    try:
        with open(path, "r", encoding="utf-8") as f:
            # quick check: if file looks like JSONL (many lines each starting with {"messages")
            sample = f.read(1000)
            f.seek(0)
            if "\n" in sample and '"messages"' in sample:
                # treat as JSONL or 1-line JSONL
                for line in f:
                    line = line.strip()
                    if not line:
                        continue
                    try:
                        obj = json.loads(line)
                        if isinstance(obj, dict) and "messages" in obj:
                            for m in obj["messages"]:
                                if isinstance(m, dict) and "role" in m and "content" in m:
                                    msgs.append({"role": m["role"], "content": m["content"]})
                        else:
                            # fallback: try extract role/content inside this line
                            msgs += extract_messages_from_text(line)
                    except Exception:
                        # fallback to regex extraction on the line
                        msgs += extract_messages_from_text(line)
            else:
                # not JSONL-like -> read whole file and extract
                full = f.read()
                msgs = extract_messages_from_text(full)
    except Exception as e:
        print("Error reading", path, e)
    return msgs


In [ ]:
# @title # CELL 3 — Xử lý dữ liệu theo logic clear.py
import os
import json
import re
from glob import glob

# --- CẤU HÌNH ĐƯỜNG DẪN ---
# Hãy đảm bảo RAW_DIR trỏ đúng tới thư mục chứa các file topic_*.jsonl trên Drive
RAW_DIR = "/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data"
CLEAN_DIR = "/content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly"
os.makedirs(CLEAN_DIR, exist_ok=True)

# --- HÀM TỪ FILE CLEAR.PY ---
def extract_messages_safely(text):
    """
    Trích tất cả messages từ file bằng Regex (logic từ clear.py)
    """
    pattern = r'"role"\s*:\s*"([^"]+)"\s*,\s*"content"\s*:\s*"((?:[^"\\]|\\.)*)"'
    matches = re.findall(pattern, text, flags=re.DOTALL)

    messages = []
    for role, content in matches:
        # Xử lý unescape ký tự unicode nếu cần, và xóa xuống dòng thừa
        # Logic gốc của clear.py: content.replace('\n', ' ').strip()
        cleaned_content = content.replace('\\n', ' ').replace('\n', ' ').strip()

        # Chuẩn hóa role cho Llama 3 (user/assistant)
        # clear.py giữ nguyên role gốc, nhưng khi train cần chuẩn hóa
        role_norm = "user" if "user" in role.lower() else "assistant"

        messages.append({
            "role": role_norm,
            "content": cleaned_content
        })
    return messages

def group_into_6pair_dialogues(messages):
    """
    Gom thành các hội thoại, mỗi hội thoại chứa 6 cặp (12 messages).
    """
    dialogues = []
    buffer = []
    pair_count = 0

    for msg in messages:
        buffer.append(msg)

        # đủ 2 messages (1 user, 1 assistant) = 1 cặp
        if len(buffer) % 2 == 0:
            pair_count += 1

        # đủ 6 cặp = 12 messages
        if pair_count == 6:
            dialogues.append({"messages": buffer.copy()})
            buffer.clear()
            pair_count = 0

    return dialogues

# --- THỰC THI ---
files = sorted(glob(f"{RAW_DIR}/*.jsonl"))
print(f"📁 Tìm thấy {len(files)} files raw.")

all_dialogues = []

for f in files:
    try:
        with open(f, "r", encoding="utf-8") as fr:
            text = fr.read()

        # 1. Trích xuất
        msgs = extract_messages_safely(text)

        # 2. Gom nhóm
        dialogs = group_into_6pair_dialogues(msgs)

        if len(dialogs) > 0:
            all_dialogues.extend(dialogs)
            print(f"  ✔ {os.path.basename(f)}: Extracted {len(msgs)} msgs -> {len(dialogs)} dialogues")
        else:
            print(f"  ⚠️ {os.path.basename(f)}: Không tạo được hội thoại nào (Messages: {len(msgs)})")

    except Exception as e:
        print(f"  ❌ Lỗi đọc file {f}: {e}")

print(f"\n📊 TỔNG CỘNG: {len(all_dialogues)} hội thoại sạch (mỗi hội thoại 12 tin nhắn).")

# Lưu file kết quả
clean_file = os.path.join(CLEAN_DIR, "dataset_cleaned.jsonl")
with open(clean_file, "w", encoding="utf-8") as fw:
    for d in all_dialogues:
        fw.write(json.dumps(d, ensure_ascii=False) + "\n")

print(f"💾 Đã lưu dataset sạch tại: {clean_file}")

Found raw files: 50
 -> /content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_01_raw_6pairs.jsonl : extracted 1056 messages
    → 88 dialogues from file
 -> /content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_02_raw_6pairs.jsonl : extracted 492 messages
    → 41 dialogues from file
 -> /content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_03_raw_6pairs.jsonl : extracted 576 messages
    → 48 dialogues from file
 -> /content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_04_raw_6pairs.jsonl : extracted 552 messages
    → 46 dialogues from file
 -> /content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_05_raw_6pairs.jsonl : extracted 504 messages
    → 42 dialogues from file
 -> /content/drive/MyDrive/DoctorAI - Trợ lý Sức khỏe Cá nhân Thông minh/tam_ly/raw_data/topic_06_raw_6pairs.jsonl : extracted 600 mes

In [ ]:
# ============================
# CELL 4 — Quick validation & stats
# ============================
from collections import Counter
lens = [len(d["messages"]) for d in all_dialogues]
print("Dialogues:", len(all_dialogues))
print("Messages per dialog - min/avg/max:", min(lens), sum(lens)/len(lens), max(lens))
# check sample
print("Sample dialogue 0:", all_dialogues[0])


Dialogues: 959
Messages per dialog - min/avg/max: 12 12.0 12
Sample dialogue 0: {'messages': [{'role': 'user', 'content': "Trá»\x9di Æ¡i, tao má»\x87t quÃ¡ mÃ\xa0y Æ¡i. Tuáº§n nÃ\xa0y ngÃ\xa0y nÃ\xa0o cÅ©ng 10 giá»\x9d Ä\x91Ãªm má»\x9bi vá»\x81. Ã\x94ng sáº¿p tao cá»© báº£o 'cá»\x91 thÃªm chÃºt ná»¯a', mÃ\xa0 cá»\x91 thÃªm lÃ\xa0 khÃ´ng cÃ³ tiá»\x81n thÃªm Ä\x91Ã³ trá»\x9di!"}, {'role': 'assistant', 'content': "Ã\x94i giá»\x9di Æ¡i, Ä\x91á»\x93ng Ä\x91á»\x99i cá»§a nhá»¯ng 'ngÆ°á»\x9di Ä\x91Æ°á»£c chá»\x8dn' Ä\x91Ã¢y rá»\x93i! Tao hiá»\x83u mÃ\xa0, cáº£m giÃ¡c y nhÆ° bá»\x8b hÃºt cáº¡n nÄ\x83ng lÆ°á»£ng Ä\x91á»\x83 lÃ\xa0m pin cho Ä\x91Ã¨n pin cá»§a cÃ´ng ty váº\xady. Sáº¿p mÃ\xa0y tÆ°á»\x9fng mÃ\xa0y lÃ\xa0 siÃªu nhÃ¢n Ã\xa0, hay lÃ\xa0 mÃ¡y cháº¡y báº±ng khÃ´ng khÃ\xad? Rá»\x93i sao ná»¯a? áº¢nh báº¯t mÃ\xa0y lÃ\xa0m cÃ¡i gÃ¬ mÃ\xa0 kinh khá»§ng dá»¯ váº\xady?"}, {'role': 'user', 'content': "ToÃ\xa0n máº¥y cÃ¡i task khÃ´ng tÃªn, deadline thÃ¬ trÃªn trá»\x9di. LÃºc tao há»\x8fi lÆ°Æ¡n

In [ ]:
# @title # CELL 5 — Tokenize & Prepare Dataset
from datasets import Dataset

# Load Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tạo Dataset từ list dictionary đã xử lý ở Cell 3
hf_dataset = Dataset.from_list(all_dialogues)

def formatting_prompts_func(example):
    """
    Sử dụng chat template chuẩn của model (Llama-3).
    Input: list of dicts [{'role': 'user', 'content': '...'}, ...]
    Output: String đã được format với special tokens (<|start_header_id|>, v.v.)
    """
    msgs = example["messages"]

    # apply_chat_template sẽ tự động thêm các token hệ thống của Llama 3
    # tokenize=False để trả về string trước, sau đó tokenizer sẽ xử lý sau
    text = tokenizer.apply_chat_template(msgs, tokenize=False, add_generation_prompt=False)

    return {"text": text}

# Bước 1: Format thành text string chuẩn Llama 3
hf_dataset = hf_dataset.map(formatting_prompts_func, batched=False)

# Bước 2: Tokenize
def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length", # Pad để batch training ổn định hơn
    )

tokenized_datasets = hf_dataset.map(tokenize_function, batched=True, remove_columns=["messages", "text"])

# Gán labels = input_ids (cho Causal LM training)
tokenized_datasets = tokenized_datasets.map(lambda x: {"labels": x["input_ids"]}, batched=True)

print("✅ Đã tokenize xong. Cấu trúc mẫu:", tokenized_datasets[0].keys())

Map:   0%|          | 0/959 [00:00<?, ? examples/s]

Columns after tokenization: ['input_ids', 'attention_mask', 'labels']
Example keys: dict_keys(['input_ids', 'attention_mask', 'labels'])


In [ ]:
# ============================
# CELL 6 — Load model (4-bit via BitsAndBytesConfig) + apply LoRA
# ============================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, # Changed to 4-bit quantization
    bnb_4bit_quant_type="nf4", # Recommended type for 4-bit
    bnb_4bit_compute_dtype=torch.bfloat16, # Use bfloat16 for computation
    bnb_4bit_use_double_quant=True, # Double quantization for further memory savings
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map={"": 0}, # Explicitly map to GPU 0
    quantization_config=bnb_config,
    trust_remote_code=True
)

# LoRA config (tuneable)
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, peft_config)

# Enable gradient checkpointing for input gradients with PEFT models
model.enable_input_require_grads()

print("Model loaded and LoRA applied.")


Model loaded and LoRA applied.


In [ ]:
# ============================
# CELL 7 — Data collator & TrainingArguments
# ============================
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    padding=True,
    max_length=MAX_LENGTH,
    label_pad_token_id=-100,
    return_tensors="pt",
)

training_args = TrainingArguments(
    output_dir="/content/output_model",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=50,
    save_steps=500,
    bf16=False,
    fp16=True,
    optim="paged_adamw_8bit",
    remove_unused_columns=False,
    gradient_checkpointing=True, # Enabled gradient checkpointing to save memory
    report_to="none"  # set "wandb" to enable
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=hf_dataset,
    data_collator=data_collator,
)

print("Trainer ready.")


Trainer ready.


In [ ]:
# ============================
# CELL 8 — Sanity checks before train
# ============================
# show a sample tensorization
sample = hf_dataset[0]
print("sample input_ids len:", len(sample["input_ids"]))
print("sample labels len:", len(sample["labels"]))
# check for empty samples
bad = [i for i, x in enumerate(hf_dataset) if len(x["input_ids"]) == 0]
print("bad examples count:", len(bad))


sample input_ids len: 1024
sample labels len: 1024
bad examples count: 0


In [ ]:
# @title # CELL 9 — Bắt đầu Training & Lưu Model
import torch

# 1. Bắt đầu train
print("🚀 Đang bắt đầu training... (Bạn có thể đi pha cà phê ☕)")
trainer.train()

# 2. Lưu model LoRA và Tokenizer xuống đĩa
OUTPUT_DIR = "/content/lora_model"
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("="*50)
print(f"✅ Đã huấn luyện xong!")
print(f"💾 Model LoRA và Tokenizer đã được lưu tại: {OUTPUT_DIR}")
print("="*50)

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:2779: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


Step,Training Loss
50,2.578800
100,2.133900
150,2.035300
200,1.981900
250,1.919700
300,1.917800
350,1.894100


Saved LoRA model to /content/lora_model


In [ ]:
# @title # CELL 10 — Test Model & Upload lên Hugging Face
from transformers import pipeline, TextStreamer
from huggingface_hub import login, HfApi

# --- PHẦN 1: TEST MODEL NHANH ---
print("🧪 Đang test thử model vừa train...")

# Sử dụng model đang có trong bộ nhớ (trainer.model) để test ngay
# Thiết lập streamer để chữ hiện ra dần dần (giống ChatGPT)
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

def test_chat(prompt_text):
    # Tạo định dạng tin nhắn theo chuẩn Llama 3
    messages = [
        {"role": "user", "content": prompt_text}
    ]

    # Format input
    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to("cuda")

    print(f"\nUser: {prompt_text}")
    print("Assistant: ", end="")

    # Generate
    _ = model.generate(
        inputs,
        max_new_tokens=256,
        streamer=streamer,
        pad_token_id=tokenizer.eos_token_id,
        temperature=0.7, # Chỉnh độ sáng tạo (0.1 - 1.0)
        do_sample=True,
        top_p=0.9
    )

# --- THỬ NGHIỆM VỚI 1 CÂU HỎI MẪU ---
test_chat("Sếp bắt tao làm thêm giờ mà không trả lương, tao phải làm sao?")




Device set to use cuda:0


KeyboardInterrupt: 

In [ ]:
# --- PHẦN 2: UPLOAD LÊN HUGGING FACE ---
# Bỏ comment các dòng dưới đây khi bạn muốn upload

# print("\n" + "="*30)
# print("☁️ BẮT ĐẦU UPLOAD LÊN HUGGING FACE")
# print("="*30)

# 1. Đăng nhập (Sẽ hiện khung nhập Token nếu chưa login)
# login()

# 2. Đặt tên Repo của bạn (Thay 'YourUsername' bằng tên user HF của bạn)
# HF_USERNAME = "YourUsername"
# REPO_NAME = "funny-empathetic-companion-v1"
# FULL_REPO_ID = f"{HF_USERNAME}/{REPO_NAME}"

# print(f"ang đẩy model lên: {FULL_REPO_ID} ...")

# 3. Push model (LoRA Adapter) & Tokenizer
# model.push_to_hub(FULL_REPO_ID, use_auth_token=True)
# tokenizer.push_to_hub(FULL_REPO_ID, use_auth_token=True)

# print(f"✅ Đã upload thành công! Link: https://huggingface.co/{FULL_REPO_ID}")